# Inferential and descriptive statistics

### Importing necessary modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid", color_codes=True)
np.random.seed(sum(map(ord, "categorical")))
matplotlib.style.use('ggplot')


In [2]:
%matplotlib inline

### Data preparation and preprocessing

The dataset was downloaded as an excel spreadsheet from the GTD website, uploaded into Pandas and then we carry out some data wrangling and preprocessing.

In [3]:
file= r'C:\Users\dejavu\Desktop\git_jupyter\springboard_mini_project\capstone_projects/globalterrorismdb_0617dist.xlsx'
df= pd.read_excel(file)

In [4]:
df.shape

(170350, 135)

In [5]:
#restrict this dataset to occurrences in the US.
df1= df['country_txt'].str.contains('United States')
df2= df[df1]
df2.head(3)

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
5,197001010002,1970,1,1,NaN,0,NaT,217,United States,1,...,"The Cairo Chief of Police, William Petersen, r...","""Police Chief Quits,"" Washington Post, January...","""Cairo Police Chief Quits; Decries Local 'Mili...","Christopher Hewitt, ""Political Violence and Te...",Hewitt Project,-9,-9,0,-9,NaN
7,197001020002,1970,1,2,NaN,0,NaT,217,United States,1,...,"Damages were estimated to be between $20,000-$...",Committee on Government Operations United Stat...,"Christopher Hewitt, ""Political Violence and Te...",NaN,Hewitt Project,-9,-9,0,-9,NaN
8,197001020003,1970,1,2,NaN,0,NaT,217,United States,1,...,The New Years Gang issue a communiqué to a loc...,"Tom Bates, ""Rads: The 1970 Bombing of the Army...","David Newman, Sandra Sutherland, and Jon Stewa...","The Wisconsin Cartographers' Guild, ""Wisconsin...",Hewitt Project,0,0,0,0,NaN


In [6]:
df2.shape

(2758, 135)

In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2758 entries, 5 to 169902
Columns: 135 entries, eventid to related
dtypes: datetime64[ns](1), float64(53), int64(24), object(57)
memory usage: 2.9+ MB


In [8]:
class wr_ppro():
    '''Used for data wrangling and some initial preprocessing'''
    def __init__(self):
        ''''''
    def drop_col_nan(self, x, threshold):
        for col in x.columns:
            amt = sum(x[col].isnull())/float(len(x)) * 100
            if amt > threshold:
                x = x.drop(col,1)
                pd.set_option('display.max_columns', None)
        return x
        
    def drop_noisy_col(self, y, w=[]):
        y=y.drop(w, 1)
        return y
                
    def drop_col_txt(self, z):
        for c in z.columns:
            c = str(c)
            if c[-3:] =='txt':
                z = z.drop(c, 1)
                pd.set_option('display.max_columns', None)
        return z     
       

In [9]:
df3=wr_ppro().drop_col_nan(df2, 80)

In [10]:
df3.index = range(len(df2))

In [11]:
df4=wr_ppro().drop_col_txt(df3)

In [12]:
df5=wr_ppro().drop_noisy_col(df4, ['corp1', 'motive', 'target1', 'weapdetail','country','addnotes', 'summary', 'scite1' , 'scite2' , 'scite3' , 'dbsource','longitude','specificity', 'eventid', 'location','region', 'propcomment', 'latitude'])

In [13]:
df5.shape

(2758, 41)

In [14]:
df5.isnull().sum()

iyear              0
imonth             0
iday               0
extended           0
provstate          0
city               0
vicinity           0
crit1              0
crit2              0
crit3              0
doubtterr          0
multiple           0
success            0
suicide            0
attacktype1        0
targtype1          0
targsubtype1     120
natlty1            9
gname              0
guncertain1        0
individual         0
nperps           982
nperpcap        1055
claimed         1051
weaptype1          0
weapsubtype1     259
nkill             73
nkillus          953
nkillter        1004
nwound            93
nwoundus         973
nwoundte        1018
property           0
propextent      1191
propvalue       1840
ishostkid        176
ransom           588
INT_LOG            0
INT_IDEO           0
INT_MISC           0
INT_ANY            0
dtype: int64

In [15]:
df5.dtypes

iyear             int64
imonth            int64
iday              int64
extended          int64
provstate        object
city             object
vicinity          int64
crit1             int64
crit2             int64
crit3             int64
doubtterr         int64
multiple          int64
success           int64
suicide           int64
attacktype1       int64
targtype1         int64
targsubtype1    float64
natlty1         float64
gname            object
guncertain1     float64
individual        int64
nperps          float64
nperpcap        float64
claimed         float64
weaptype1         int64
weapsubtype1    float64
nkill           float64
nkillus         float64
nkillter        float64
nwound          float64
nwoundus        float64
nwoundte        float64
property          int64
propextent      float64
propvalue       float64
ishostkid       float64
ransom          float64
INT_LOG           int64
INT_IDEO          int64
INT_MISC          int64
INT_ANY           int64
dtype: object

***Imputing Missing Values***

In [16]:
from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.
        Columns of dtype object are imputed with the most frequent value 
        in column.
        Columns of other types are imputed with median of column.
        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
            index=X.columns)
        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [17]:
df6 = DataFrameImputer().fit_transform(df5)
df6.head()

,iyear,imonth,iday,extended,provstate,city,vicinity,crit1,crit2,crit3,doubtterr,multiple,success,suicide,attacktype1,targtype1,targsubtype1,natlty1,gname,guncertain1,individual,nperps,nperpcap,claimed,weaptype1,weapsubtype1,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent,propvalue,ishostkid,ransom,INT_LOG,INT_IDEO,INT_MISC,INT_ANY
0,1970,1,1,0,Illinois,Cairo,0,1,1,1,0,0,1,0,2,3,22.0,217.0,Black Nationalists,0.0,0,-99.0,-99.0,0.0,5,5.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3.0,5000.0,0.0,0.0,-9,-9,0,-9
1,1970,1,2,0,California,Oakland,0,1,1,1,1,0,1,0,3,21,107.0,217.0,Unknown,0.0,0,-99.0,-99.0,0.0,6,16.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3.0,22500.0,0.0,0.0,-9,-9,0,-9
2,1970,1,2,0,Wisconsin,Madison,0,1,1,1,0,0,1,0,7,4,28.0,217.0,New Year's Gang,0.0,0,1.0,1.0,1.0,8,19.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3.0,60000.0,0.0,0.0,0,0,0,0
3,1970,1,3,0,Wisconsin,Madison,0,1,1,1,0,0,1,0,7,2,21.0,217.0,New Year's Gang,0.0,0,1.0,1.0,0.0,8,20.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3.0,5000.0,0.0,0.0,0,0,0,0
4,1970,1,1,0,Wisconsin,Baraboo,0,1,1,0,1,0,0,0,3,4,27.0,217.0,"Weather Underground, Weathermen",0.0,0,-99.0,-99.0,0.0,6,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0,3.0,0.0,0.0,0.0,0,0,0,0


***Go to data_science_capstone_project_1 (2) data_storytelling.ipynb in the same directory for the exploratory data analysis***

** Removing the unknowns from group name (gname) column**

In [18]:
df7 = df6[df6['gname'] != 'Unknown']
df7.shape

(2206, 41)

In [19]:
# this will be useful in future analysis as an out of sample dataset 
df8 = df6[df6['gname'] == 'Unknown']
df8.shape

(552, 41)

**Reseting the Index**


In [20]:
df9 = df7.reset_index(drop=True)
df9.head()

,iyear,imonth,iday,extended,provstate,city,vicinity,crit1,crit2,crit3,doubtterr,multiple,success,suicide,attacktype1,targtype1,targsubtype1,natlty1,gname,guncertain1,individual,nperps,nperpcap,claimed,weaptype1,weapsubtype1,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent,propvalue,ishostkid,ransom,INT_LOG,INT_IDEO,INT_MISC,INT_ANY
0,1970,1,1,0,Illinois,Cairo,0,1,1,1,0,0,1,0,2,3,22.0,217.0,Black Nationalists,0.0,0,-99.0,-99.0,0.0,5,5.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3.0,5000.0,0.0,0.0,-9,-9,0,-9
1,1970,1,2,0,Wisconsin,Madison,0,1,1,1,0,0,1,0,7,4,28.0,217.0,New Year's Gang,0.0,0,1.0,1.0,1.0,8,19.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3.0,60000.0,0.0,0.0,0,0,0,0
2,1970,1,3,0,Wisconsin,Madison,0,1,1,1,0,0,1,0,7,2,21.0,217.0,New Year's Gang,0.0,0,1.0,1.0,0.0,8,20.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3.0,5000.0,0.0,0.0,0,0,0,0
3,1970,1,1,0,Wisconsin,Baraboo,0,1,1,0,1,0,0,0,3,4,27.0,217.0,"Weather Underground, Weathermen",0.0,0,-99.0,-99.0,0.0,6,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0,3.0,0.0,0.0,0.0,0,0,0,0
4,1970,1,6,0,Colorado,Denver,0,1,1,1,1,0,1,0,7,4,28.0,217.0,Left-Wing Militants,0.0,0,-99.0,-99.0,0.0,8,19.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3.0,305.0,0.0,0.0,-9,-9,0,-9


***Checking for collinearity between predictive variables and between predictive and target variable pairs***

In [28]:
df9.corr()

,iyear,imonth,iday,extended,vicinity,crit1,crit2,crit3,doubtterr,multiple,success,suicide,attacktype1,targtype1,targsubtype1,natlty1,guncertain1,individual,nperps,nperpcap,claimed,weaptype1,weapsubtype1,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent,propvalue,ishostkid,ransom,INT_LOG,INT_IDEO,INT_MISC,INT_ANY
iyear,1.000000,0.076550,-0.049476,0.031092,0.044660,-0.171872,-0.016508,0.016371,-0.031846,0.053408,0.006521,0.097549,0.244447,0.100240,0.134811,0.002343,-0.020429,0.494495,0.363108,0.557197,0.146279,0.168770,-0.097773,0.054963,0.054925,0.155189,0.050394,0.031833,0.047183,0.034723,-0.115104,0.026387,0.011992,-0.024071,-0.100307,-0.076742,-0.025972,-0.067971
imonth,0.076550,1.000000,0.032484,-0.027926,-0.006559,0.033956,-0.026647,-0.018949,0.032496,0.027567,0.016136,0.016664,-0.038614,-0.011569,-0.001163,0.004684,-0.031102,0.003451,0.048539,0.048307,-0.070830,-0.002356,-0.046056,0.025124,0.025185,0.026748,0.024656,0.008527,0.018777,0.000516,0.015668,-0.015366,0.007605,-0.004499,0.095267,0.097389,0.043200,0.093909
iday,-0.049476,0.032484,1.000000,-0.007623,0.001623,0.031525,-0.039694,0.034405,0.003414,-0.042806,0.000265,-0.014050,-0.016123,-0.005040,-0.018401,-0.026119,0.001992,-0.094972,-0.014719,-0.064770,-0.024700,0.015337,-0.001681,-0.015385,-0.015714,-0.035879,-0.013191,0.011597,0.015074,0.039005,-0.006757,0.007455,-0.005191,0.023772,0.096798,0.092384,0.038975,0.097040
extended,0.031092,-0.027926,-0.007623,1.000000,0.050697,0.005296,0.010330,0.006852,0.005069,-0.029383,0.028826,-0.003611,0.050561,0.022492,0.014522,0.007028,-0.021836,0.005008,0.109452,0.056887,0.025030,0.000368,-0.042485,-0.001962,-0.001885,0.050265,-0.002156,-0.001439,0.038309,-0.028362,-0.014058,0.000023,0.217819,0.122836,0.017141,0.017508,-0.010498,0.015494
vicinity,0.044660,-0.006559,0.001623,0.050697,1.000000,0.010197,-0.003853,-0.092644,0.026188,-0.036877,0.025580,0.059445,0.061046,0.023107,0.018165,0.022074,0.043495,-0.012232,0.021805,0.041303,-0.035469,0.075376,-0.001393,-0.000486,-0.001061,0.047116,-0.004005,-0.004577,-0.009513,0.008705,-0.059378,0.011510,0.076097,0.059445,0.022164,0.029047,-0.004858,0.025216
crit1,-0.171872,0.033956,0.031525,0.005296,0.010197,1.000000,-0.013354,-0.008859,-0.070017,-0.063850,0.006932,0.004668,-0.061847,-0.059584,-0.049141,0.003388,0.028229,-0.168033,-0.061359,-0.132082,-0.043592,0.070682,0.045359,0.002405,0.002301,-0.013628,0.002820,0.002463,-0.127800,0.019413,-0.013645,0.002290,-0.022148,0.004668,0.062823,0.063241,0.002231,0.052969
crit2,-0.016508,-0.026647,-0.039694,0.010330,-0.003853,-0.013354,1.000000,-0.017279,-0.146713,0.074091,-0.026503,0.009106,-0.017567,0.109623,0.148445,0.007147,-0.001522,0.050397,-0.148133,-0.116949,0.078000,0.021998,0.252617,0.005680,0.005507,0.016346,0.005786,0.007420,0.012458,-0.041072,0.018824,0.002894,-0.122063,0.009106,-0.109500,-0.107495,0.008696,-0.101797
crit3,0.016371,-0.018949,0.034405,0.006852,-0.092644,-0.008859,-0.017279,1.000000,-0.100743,0.037857,-0.002468,0.006041,0.057158,0.036121,0.023651,-0.011757,0.036527,0.018381,0.035753,0.056814,0.003558,0.048184,0.046717,0.002398,0.002557,0.010844,0.002892,-0.003429,-0.017780,-0.003077,0.011164,-0.011449,0.017120,0.006041,-0.071702,-0.066290,0.017560,-0.065362
doubtterr,-0.031846,0.032496,0.003414,0.005069,0.026188,-0.070017,-0.146713,-0.100743,1.000000,-0.022704,-0.009451,0.006287,-0.028223,-0.000678,-0.026315,0.002278,0.030703,-0.115620,0.011952,0.046606,0.000289,-0.024937,-0.043488,-0.001034,-0.001027,0.003942,-0.001242,-0.000789,0.011400,-0.057710,0.037747,0.001048,-0.037838,-0.067354,-0.021211,-0.020729,-0.055651,-0.019685
multiple,0.053408,0.027567,-0.042806,-0.029383,-0.036877,-0.063850,0.074091,0.037857,-0.022704,1.000000,-0.071030,0.059127,-0.040506,0.037807,0.031324,-0.020249,0.054627,0.054129,0.005035,0.061554,-0.028042,-0.021265,-0.000176,0.068640,0.068871,0.058442,0.065166,-0.015040,0.008187,-0.012545,-0.087042,-0.011443,-0.004400,-0.025902,-0.000643,0.000562,-0.043347,-0.001962


From the above statistics, we see there is collinearity between targsubtype1 and targtype1, and the variables nkill, nkillus, and nwound. There is also similar issue between INT_LOG, INT_IDEO, INT_MISC and INT_ANY. Looking at pairwise relationship between each predictor with the target variable, we see collinearity issue between INT_LOG and INT_ANY and the target variable - INT_IDEO. Usually, any pairwise collinearity between predictors above 0.8 is regarded high and it is advisable to drop one of such with a higher collinearity with respect to the target variable. It is usually the case when dealing with linear models, but it is known that more advanced models correct for such. So, i will leave all predictors in there to see how my model perform with and without them.

***Hot coding categorical variables***

From cell 15, we see that gname, provstate and city are identified as text based categorical variables. These need converted to numerical variable because Random Forest (RF) cannot handle string or text. One way to do that is to use Label Encoding module in scikit learn library. However, with that you now run into another issue that can best be explained here: https://stackoverflow.com/questions/44959122/how-to-handle-categorical-features-for-decision-tree-random-forest-in-spark-ml and https://www.analyticsvidhya.com/blog/2016/07/practical-guide-data-preprocessing-python-scikit-learn/
RF should be able to handle categorical features when encoding as numerical arrays. In fact RF implementation in R does to about 32 cardinality depth, with options for more. However, as of this analysis, RF implemenation in scikit learn will give a misleading interpretaion of the encoded numerical arrays. For such features, It is advisable to use One-Hot Encoding module in scikit learn after label encoding when categories > 2 to provide optimum result.

Instead of going this route, Pandas also has a function for such that can handle both computations, making it a little shorter. It also allows me see the generated dummy features in nice format.

In [21]:
var_mod = ['gname', 'provstate', 'city']

#create the dummy variables for each category of the three features 
abc = pd.get_dummies(df9[var_mod])

#drop original features once dummy variables have been created
dfg = df9.drop(['gname', 'provstate', 'city'], axis=1)

In [22]:
abc.shape

(2206, 873)

In [23]:
#merge the created dummy variables with the original dataframe and reset the index for the overall dataframe
df10 = pd.concat([dfg, abc], axis=1).reset_index(drop=True)
df10.shape

(2206, 911)

In [24]:
df10.head()

,iyear,imonth,iday,extended,vicinity,crit1,crit2,crit3,doubtterr,multiple,success,suicide,attacktype1,targtype1,targsubtype1,natlty1,guncertain1,individual,nperps,nperpcap,claimed,weaptype1,weapsubtype1,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent,propvalue,ishostkid,ransom,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,gname_Action Squad,gname_African-American extremists,gname_Al-Qaida,gname_Al-Qaida in the Arabian Peninsula (AQAP),gname_American Indian Movement,gname_American Servicemen's Union (ASU),gname_Americans for Justice,gname_Americans for a Competent Federal Judicial System,gname_Anarchists,gname_Animal Liberation Front (ALF),gname_Animal Rights extremists,gname_Anti Shah of Iran,gname_Anti-Abortion extremists,gname_Anti-Castro Command,gname_Anti-Castro Group,gname_Anti-Communist Viets Organization,gname_Anti-Environmentalists,gname_Anti-Government Group,gname_Anti-Government extremists,gname_Anti-Gun Control extremists,gname_Anti-Israeli extremists,gname_Anti-Kim Jong-il extremists,gname_Anti-Liberal extremists,gname_Anti-Muslim extremists,gname_Anti-Nuclear extremists,gname_Anti-Police extremists,gname_Anti-Semitic extremists,gname_Anti-Sikh extremists,gname_Anti-Technology extremists,gname_Anti-Trump extremists,gname_Anti-White extremists,gname_Antonia Martinez Student Commandos (AMSC),gname_Arabs,gname_Armed Commandos of Liberation,gname_Armed Commandos of Student Self Defense,gname_Armed Forces of Popular Resistance (FARP),gname_Armed Revolutionary Independence Movement (MIRA),gname_Armenian Group,gname_Armenian Secret Army for the Liberation of Armenia,gname_Armenians,gname_Army of God,gname_Aryan Nation,gname_Aryan Republican Army,gname_BAY Bombers,gname_Black Afro Militant Movement,gname_Black American Moslems,gname_Black Brigade (United States),gname_Black Hebrew Israelites,gname_Black Liberation Army,gname_Black Muslims,gname_Black Nationalists,gname_Black Panthers,gname_Black Revolutionary Assault Team,gname_Black September,gname_Boricua Revolutionary Front,gname_Chicano Liberation Front,gname_Chicano Radicals,gname_Christian Liberation Army,gname_Citizens for Constitutional Freedom,gname_Coalition to Save the Preserves (CSP),gname_Comrades in Arms,gname_Condor,gname_Continental Revolutionary Army,gname_Coordination of the United Revolutionary Organization (CORU),gname_Court Reform extremists,"gname_Covenant, Sword and the Arm of the Lord (CSA)",gname_Croatian Freedom Fighters,gname_Croatian Liberation Army,gname_Croatian Nationalists,gname_Cuban Action,gname_Cuban C-4 Movement,gname_Cuban Exiles,gname_Cuban Secret Army,gname_Cuban extremists,gname_Cypriot,gname_Earth First!,gname_Earth Liberation Front (ELF),gname_Earth Night Action Group,gname_East Side Action Committee,gname_El Movimiento Revolucionario Independinista,gname_Environmental Life Force,gname_Environmentalists,gname_Evan Mecham Eco-Terrorist International Conspiracy (EMETIC),gname_Farm Animal Revenge Militia (FARM),gname_Fourth Reich Skinheads,gname_Fred Hampton Unit of the People's Forces,gname_Front for the National Liberation of Puerto Rico,gname_Fuerzas Armadas de Liberacion Nacional (FALN),gname_Gaddafi loyalists,gname_Gay Liberation Front,gname_George Jackson Brigade,gname_Grupo Estrella,gname_Guerrilla Forces for Liberation,gname_Hanafi Muslims,gname_Hatikvah Leumi or National Hope,gname_Hawaiian extremists,gname_Imperial Iranian Patriotic Organization,gname_Independent Armed Revolutionary Commandos (CRIA),gname_International Committee Against Nazism,gname_Iranian extremists,gname_Iranians,gname_Iraqi extremists,gname_Irish Republican Army (IRA),gname_Islamist extremists,gname_Jamaat-al-Fuqra,gname_Jewish Armed Resistance,gname_Jewish Committee of Concern,gname_Jewish Defenders,gname_Jewish Defense League (JDL),gname_Jewish Direct Action,gname_Jewish Extremists,gname_Jihadi-inspired extremists,gname_Jonathan Jackson Brigade,gname_Justice Commandos for the Armenian Genocide,gname_Kahane Chai,gname_Ku Klux Klan,gname_Latin Am

An issue with encoding data this way is greatly increasing the dimensionality of the dataset. This datadet now has 910 features compared with 2206 observations, which may lead to what is called curse of dimensionality - a phenomenon that causes some learning algorithms to perform poorly. I like the explanation that William Chen gave https://www.quora.com/What-is-the-curse-of-dimensionality. 

Since i am not doing unsupervised clustering, a way i propose to get around this is using RF to select important variables. This will prune the dimension down to one that can provide good accuracy.

In [122]:
df11 = df10.sample(frac=0.85, replace=True)
df12 = df11[(df11['INT_LOG'] != -9) & (df11['INT_MISC'] !=-9) & (df11['INT_ANY'] != -9) & (df11['INT_IDEO'] != -9)]
df12.shape

(1185, 911)

### Descriptive statistics 

We start off this analysis by computing the descriptive statistics that summarizes the dataset in terms of measures of central tendency, dispersion and shape of the dataset’s distribution

In [25]:
df10.describe()

iyear       imonth         iday     extended     vicinity  \
count  2206.000000  2206.000000  2206.000000  2206.000000  2206.000000   
mean   1981.915231     6.273345    15.424751     0.004080     0.014959   
std      12.528338     3.428451     9.165770     0.063757     0.121417   
min    1970.000000     1.000000     0.000000     0.000000     0.000000   
25%    1971.000000     3.000000     7.000000     0.000000     0.000000   
50%    1977.000000     6.000000    15.000000     0.000000     0.000000   
75%    1988.000000     9.000000    23.000000     0.000000     0.000000   
max    2016.000000    12.000000    31.000000     1.000000     1.000000   

             crit1        crit2        crit3    doubtterr     multiple  \
count  2206.000000  2206.000000  2206.000000  2206.000000  2206.000000   
mean      0.993200     0.974615     0.988667     0.033092     0.174071   
std       0.082198     0.157328     0.105874     0.985285     0.379256   
min       0.000000     0.000000     0.000000    -9.000000     0.000000   
25%       1.000000     1.000000     1.000000     0.000000     0.000000   
50%       1.000000     1.000000     1.000000     0.000000     0.000000   
75%       1.000000     1.000000     1.000000     0.000000     0.000000   
max       1.000000     1.000000     1.000000     1.000000     1.000000   

           success      suicide  attacktype1    targtype1  targsubtype1  \
count  2206.000000  2206.000000  2206.000000  2206.000000   2206.000000   
mean      0.831369     0.003173     4.236627     5.850408     34.859021   
std       0.374511     0.056254     2.092041     5.485856     27.071612   
min       0.000000     0.000000     1.000000     1.000000      1.000000   
25%       1.000000     0.000000     3.000000     1.000000     11.000000   
50%       1.000000     0.000000     3.000000     4.000000     28.000000   
75%       1.000000     0.000000     7.000000     8.000000     49.000000   
max       1.000000     1.000000     9.000000    22.000000    110.000000   

           natlty1  guncertain1   individual       nperps     nperpcap  \
count  2206.000000  2206.000000  2206.000000  2206.000000  2206.000000   
mean    212.775159     0.104261     0.088849   -66.985494   -71.115594   
std      38.482841     0.305668     0.284590    47.368165    45.019500   
min       8.000000     0.000000     0.000000   -99.000000   -99.000000   
25%     217.000000     0.000000     0.000000   -99.000000   -99.000000   
50%     217.000000     0.000000     0.000000   -99.000000   -99.000000   
75%     217.000000     0.000000     0.000000     1.000000     0.000000   
max     422.000000     1.000000     1.000000   200.000000    11.000000   

           claimed    weaptype1  weapsubtype1        nkill      nkillus  \
count  2206.000000  2206.000000   2206.000000  2206.000000  2206.000000   
mean      0.043518     6.657752     15.400725     1.620127     1.532638   
std       1.309623     1.550744      5.961546    41.983344    40.741490   
min      -9.000000     1.000000      1.000000     0.000000     0.000000   
25%       0.000000     6.000000     16.000000     0.000000     0.000000   
50%       0.000000     6.000000     16.000000     0.000000     0.000000   
75%       0.000000     8.000000     18.000000     0.000000     0.000000   
max       1.000000    13.000000     30.000000  1383.000000  1358.000000   

          nkillter       nwound     nwoundus     nwoundte     property  \
count  2206.000000  2206.000000  2206.000000  2206.000000  2206.000000   
mean      0.025385     8.056664     1.035811     0.012693     0.341795   
std       0.250694   222.774358    21.368213     0.164468     2.025494   
min       0.000000     0.000000     0.000000     0.000000    -9.000000   
25%       0.000000     0.000000     0.000000     0.000000     0.000000   
50%       0.000000     0.000000     0.000000     0.000000     1.000000   
75%       0.000000     0.000000     0.000000     0.000000     1.000000   
max       5.000000  7366.000000   751.000000     4.000000     1.0000

From the descriptive statistics above, one can tell the following:
1. the skewness of each features by comparing the mean to the median (50% mark). Most of the variables, especially the dummy variables seem slightly right skewed, but nothing really serious since variance is not overly high 
2. skewness from #1 if any,  translate to some outliers in the distribution. And, outliers can sometimes affect results of the analysis depending on the machine learning model one go with and may need further investigation.
3. the distribution of the datapoints. The greater the standard deviation ('std'), the more dispersed the datapoints are.

One can delve deeper into each feature using scatterplots and/or histogram particularly if one is using linear algorithms. Considering high dimensionality of the dataset, and since my analysis is using Random Forest (RF) algorithm; we may not necessarily need to do that. Industry arguments suggest that RF is robust enough to cater for collinearlity and outlier deficiencies.

***Statistical analysis to establish significant impact of some features on nationalities of attackers***

From the exploratory data analysis already carried out, we know that INT_IDEO - nationality of attackers, has some association with timeseries, "iyear." I am going to delve deeper into this to verify if this is relationship is trully significant, and check if timeseries is the most significant feature out of the bunch. We start off with a hypothesis test and then double check our work with a linear regression model. 

The Two-Sample T-Test should be appropriate for this, because we are comparing assumed independent measures on observational units where the standard deviation is either unkown or not given.

Central Limit Theorem (CLT) also applies because the t-statistics follow a distribution which is the t-distribution. This distribution arises from a plot of each and every t-statistics of every sample randomly selected from the measured population measured. Population measured being each variable of the observational units. And, this distribution is assumed to be normal following a bell curve with the t-statics values in the horizontal and the probability density function (PDF) on the vertical. The more random samples one select from the population, the more the curve tend to a finer and perfect normal distribution. This exemplify the CLT in that even if a population that random samples are drawn from is not normal, the sample mean distribution itself, which is the plot of the sample mean from randomly selected samples from the population tends to a normal distribution. 

*** The null and alternate hypotheses***

Ho: the population mean of americans perpetrating terrorist attacks is same as foreigners between 1940 and 2016

Ha: the population mean of americans perpetrating terrorist attacks is significantly different from that of foreigners between 1940 and 2016

In [62]:
from scipy import stats
import math
import random

In [69]:
df7 = df6.sample(frac=0.85, replace=True)
df8 = df7[(df7['INT_LOG'] != -9) & (df7['INT_MISC'] !=-9) & (df7['INT_ANY'] != -9) & (df7['INT_IDEO'] != -9)]
df8.shape

(1179, 41)

In [70]:
df8.head()

,iyear,imonth,iday,extended,provstate,city,vicinity,crit1,crit2,crit3,doubtterr,multiple,success,suicide,attacktype1,targtype1,targsubtype1,natlty1,gname,guncertain1,individual,nperps,nperpcap,claimed,weaptype1,weapsubtype1,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent,propvalue,ishostkid,ransom,INT_LOG,INT_IDEO,INT_MISC,INT_ANY
863,1974,5,3,0,Connecticut,New Haven,0,1,0,1,1,0,1,0,2,1,3.0,217.0,Black Liberation Army,0.0,0,4.0,-99.0,0.0,5,3.0,0.0,0.0,0.0,1.0,0.0,0.0,1,4.0,5000.0,0.0,0.0,0,0,0,0
2042,1991,7,6,0,Puerto Rico,Borinquen,0,1,1,1,0,0,1,0,7,4,31.0,217.0,Popular Liberation Army (Puerto Rico),0.0,0,-99.0,-99.0,0.0,8,20.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2.0,5000.0,0.0,0.0,0,0,0,0
1448,1979,10,18,0,New York,New York City,0,1,1,1,0,1,0,0,3,2,15.0,217.0,Fuerzas Armadas de Liberacion Nacional (FALN),1.0,0,-99.0,-99.0,1.0,6,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0,3.0,5000.0,0.0,0.0,1,0,0,1
872,1974,6,11,0,California,Union City,0,1,1,1,0,0,1,0,2,3,25.0,217.0,Chicano Liberation Front,1.0,0,1.0,-99.0,0.0,5,4.0,1.0,1.0,0.0,4.0,4.0,4.0,-9,3.0,5000.0,0.0,0.0,0,0,0,0
1016,1975,8,5,0,Washington,Tacoma,0,1,1,1,0,1,1,0,3,2,14.0,217.0,George Jackson Brigade,0.0,0,-99.0,-99.0,0.0,6,16.0,0.0,0.0,0.0,0.0,0.0,0.0,1,4.0,5000.0,0.0,0.0,0,0,0,0


In [71]:
df9 = df8[["iyear", "INT_IDEO"]]
df9.head()

,iyear,INT_IDEO
863,1974,0
2042,1991,0
1448,1979,0
872,1974,0
1016,1975,0


In [72]:
citizen = []
for x in df8['INT_IDEO']:
    if x == 0:
        x = 'american'
    else: 
        x = 'foreigner'
    citizen.append(x)

In [186]:
df9['citizen'] = citizen
df9.head()

C:\Users\dejavu\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,iyear,INT_IDEO,citizen
863,1974,0,american
2042,1991,0,american
1448,1979,0,american
872,1974,0,american
1016,1975,0,american


In [187]:
us_grp = df9[df9.citizen =='american']['iyear']
us_cnt = us_grp.count()
f_grp = df9[df9.citizen =='foreigner']['iyear']
f_cnt = f_grp.count()
total_cnt = df9['citizen'].count()
stats.ttest_ind(us_grp, f_grp)

Ttest_indResult(statistic=-12.489583120983752, pvalue=1.0494737689813117e-33)

***Calculating the margin of error and confidence interval***

In [116]:
print('count of americans:', us_grp.count()) 
print('count of foreigners:', f_grp.count())
print('total count of known individuals', total_cnt)

count of americans: 751
count of foreigners: 428
total count of known individuals 1179


In [117]:
## americans
ci = 1.96
p = us_cnt/total_cnt
margin_of_error = np.sqrt((p*(1-p)/len(df9)))*ci
ci_lo = p - margin_of_error
ci_hi = p + margin_of_error
print("americans")
print("Mean: ", p)
print("Margin of error: ", margin_of_error)
print("Confidence interval: ", ci_lo, ',', ci_hi)

americans
Mean:  0.636980491942
Margin of error:  0.02744904358
Confidence interval:  0.609531448362 , 0.664429535522


In [118]:
## foreigners
ci = 1.96
p = f_cnt/total_cnt
margin_of_error = np.sqrt((p*(1-p)/len(df9)))*ci
ci_lo = p - margin_of_error
ci_hi = p + margin_of_error
print("foreigners")
print("Mean: ", p)
print("Margin of error: ", margin_of_error)
print("Confidence interval: ", ci_lo, ',', ci_hi)

foreigners
Mean:  0.363019508058
Margin of error:  0.02744904358
Confidence interval:  0.335570464478 , 0.390468551638


Since the p-value of the 2-sample t test is way less than the significant level α = 0.01 and the confidence interval of the portion of the both nationalities does not include zero, we reject the null hypothesis Ho, reporting that there is statistically significant difference between the overall average number of Americans perpetrating terrorist attacks compared to foreign nationals.

This does not tell the whole story, because they are summary statistics. Other factors need to be taken into consideration to make a firm conclusion. Just like tasting soup with a tea spoon after stirring when cooking, the portion you taste is the sample that was randomly selected and is reflection of the whole pot of soup. We know that timeseries is important because it implies some significance within the target variables. However, we are still not sure if other variables are also signiﬁcant or whether timeseries is the most important factor. To understand the relation between nationalities (INT_IDEO) and other variable, we could run a logistic regression test as further analysis. Considering we have a high dimensionality issue here, I have decided to leave the selection of importance selection to the machine learning model we are using. 

***Double Checking Statistical Inference***

Just to get an idea of what is going on here, I am going to cheat a little bit here. From the model i already built using the Random Forest algorithm, we already know the predictors that are the most important features that can either explain or predict the target variable (INT_IDEO). Let us double check if some of them have any statistical significance here. 

In [119]:
import statsmodels.formula.api as sm

In [183]:
X = df12[['iyear', 'imonth', 'iday', 'doubtterr', 'attacktype1', 'targtype1', 'targsubtype1', 'natlty1', 
           'nperps', 'nperpcap', 'weaptype1', 'weapsubtype1', 'propvalue', 'INT_LOG', 'INT_MISC', 'INT_ANY']] 
y = df12['INT_IDEO']

In [184]:
model = sm.Logit(y, X)

In [185]:
result = model.fit()

Optimization terminated successfully.
         Current function value: 0.131408
         Iterations 13


In [176]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               INT_IDEO   No. Observations:                 1185
Model:                          Logit   Df Residuals:                     1169
Method:                           MLE   Df Model:                           15
Date:                Sun, 08 Apr 2018   Pseudo R-squ.:                  0.8002
Time:                        23:53:30   Log-Likelihood:                -155.72
converged:                       True   LL-Null:                       -779.55
                                        LLR p-value:                9.431e-257
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
iyear           -0.0074      0.002     -3.361      0.001      -0.012      -0.003
imonth           0.0572      0.041      1.389      0.165      -0.024       0.138
iday            -0.0595      0.016     -3.654      0.000      -0.091      -0.028
doubtterr       -0.3645      0.504     -0.724      0.469      -1.351       0.622
attacktype1     -0.0578      0.146     -0.395      0.693      -0.345       0.229
targtype1       -0.1213      0.120     -1.009      0.313      -0.357       0.114
targsubtype1     0.0472      0.021      2.224      0.026       0.006       0.089
natlty1          0.0234      0.005      4.494      0.000       0.013       0.034
nperps           0.0162      0.006      2.708      0.007       0.004       0.028
nperpcap        -0.0051      0.008     -0.640      0.522      -0.021       0.011
weaptype1       -0.0714      0.231     -0.309      0.758      -0.525       0.382
weapsubtype1     0.0534      0.033      1.603      0.109      -0.012       0.119
propvalue      8.45e-08   4.76e-07      0.177      0.859   -8.49e-07    1.02e-06
INT_LOG         -6.3768      1.093     -5.832      0.000      -8.520      -4.234
INT_MISC         2.7596      0.654      4.223      0.000       1.479       4.040
INT_ANY         16.3575      4.215      3.881      0.000       8.097      24.618
================================================================================

Possibly complete quasi-separation: A fraction 0.41 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

Remember the timeseries features we worked with earlier - iyear; we see the p-value is still way less than 0.01, which confirms that iyear is indeed important and significant in either explaining or predicting nationalities of attackers (INT_IDEO). It should be expected that the more the predictors added to this model, the lesser the significance of each features in the model in relation to the dependent variable. This means the p-value of "iyear" will increase, but still be significant. This can be seen when we had just "iyear" in relation to "INT_IDEO," p-value was 1.0494737689813117e-33, but with fifteen other predictors added to the model in addition to "iyear," we see that p-value of "iyear" is now 0.001. 

Something else worth of note is that "iyear" is probably not the most significant predictor in the overall model. Some other features are slightly better. For example, we see that "iday," which is time of the day, and the "natlty1" are slightly more significant. However, i am more interested in "iyear" in this analysis. So, we should have it at the back of our mind that some other factors in the pool may also be turning the wheel.

***References***

https://stats.stackexchange.com/questions/168622/why-is-multicollinearity-not-checked-in-modern-statistics-machine-learning

https://stats.stackexchange.com/questions/141619/wont-highly-correlated-variables-in-random-forest-distort-accuracy-and-feature

http://blog.datadive.net/selecting-good-features-part-iii-random-forests/

https://stats.stackexchange.com/questions/187200/how-are-random-forests-not-sensitive-to-outliers

http://blog.yhat.com/posts/logistic-regression-and-python.html

https://mashimo.wordpress.com/2017/07/26/logistic-regression-with-python-statsmodels/

http://www.statsmodels.org/dev/

http://www.science.smith.edu/~jcrouser/SDS293/labs/2016/lab4/Lab%204%20-%20Logistic%20Regression%20in%20Python.pdf

http://slendermeans.org/ml4h-ch2-p2.html

https://stats.stackexchange.com/questions/203740/logistic-regression-scikit-learn-vs-statsmodels





